In [1]:
pip install crewai 'crewai[tools]' langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [6]:
financial_forecaster_agent = Agent(
    role="Financial Forecaster",
    goal="Develop and refine predictive models for asset pricing and risk assessment, providing accurate financial forecasts.",
    backstory="With a deep understanding of econometric techniques and financial theory, this agent crafts sophisticated models to anticipate market fluctuations. The Financial Forecaster Agent is essential for strategic planning and investment optimization, leveraging historical data and current market conditions to project future outcomes.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [7]:
risk_manager_agent = Agent(
    role="Risk Manager",
    goal="Identify, assess, and mitigate financial risks across various portfolios using advanced quantitative techniques.",
    backstory="With a keen eye for potential vulnerabilities, this agent excels in building robust risk models and conducting stress tests. The Risk Manager Agent is crucial for safeguarding investments and ensuring regulatory compliance, employing sophisticated calculations and data analysis to minimize exposure to adverse market conditions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [8]:
portfolio_optimizer_agent = Agent(
    role="Portfolio Optimizer",
    goal="Construct and rebalance investment portfolios to maximize returns while adhering to specific risk tolerances and investment objectives.",
    backstory="This agent specializes in applying optimization algorithms and financial modeling to create efficient portfolios. The Portfolio Optimizer Agent is essential for achieving optimal investment outcomes, leveraging data analysis and complex calculations to dynamically adjust asset allocations in response to changing market dynamics.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

In [9]:
financial_forecast_task = Task(
    description=(
        "Quick forecast for {stock_selection} price next quarter. Focus on key metrics only."
    ),
    expected_output=(
        "Brief price forecast with key support/resistance levels for {stock_selection}."
    ),
    agent=financial_forecaster_agent,
    timeout=120  # Reduced to 2 minutes
)

In [10]:
risk_assessment_task = Task(
    description=(
        "Quick risk assessment for {stock_selection}. List main risk factors."
    ),
    expected_output=(
        "Key risks and basic mitigation steps for {stock_selection}."
    ),
    agent=risk_manager_agent,
    timeout=120
)

In [11]:
portfolio_optimization_task = Task(
    description=(
        "Basic portfolio allocation for {stock_selection} with {initial_capital} and {risk_tolerance} risk."
    ),
    expected_output=(
        "Simple allocation strategy for {stock_selection} considering {risk_tolerance} risk."
    ),
    agent=portfolio_optimizer_agent,
    timeout=120
)

In [12]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_analysis_crew = Crew(
    agents=[financial_forecaster_agent, 
            risk_manager_agent, 
            portfolio_optimizer_agent],
    
    tasks=[financial_forecast_task, 
           risk_assessment_task, 
           portfolio_optimization_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.3),
    process=Process.hierarchical,
    verbose=True
)

In [13]:
financial_analysis_inputs = {
    'stock_selection': 'CAP.PA',
    'initial_capital': '250000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [14]:
# Add this at the beginning of your script before importing CrewAI
import os
import ssl
import urllib3
from urllib3.util.ssl_ import create_urllib3_context

# Method 1: Disable CrewAI telemetry completely
os.environ["CREWAI_DISABLE_TELEMETRY"] = "true"

# Method 2: If Method 1 doesn't work, try this more comprehensive approach
# First, disable warnings about insecure requests
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create a custom SSL context
custom_context = create_urllib3_context()
custom_context.check_hostname = False
custom_context.verify_mode = ssl.CERT_NONE

# Patch the urllib3 SSL context factory
def patched_create_urllib3_context(*args, **kwargs):
    return custom_context

# Apply the patch
urllib3.util.ssl_.create_urllib3_context = patched_create_urllib3_context

In [15]:
result = financial_analysis_crew.kickoff(inputs=financial_analysis_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Quick forecast for CAP.PA price next quarter. Focus on key metrics only.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of creating a quick forecast for CAP.PA price next quarter to the Financial Forecaster, as they are the best person for this job.

Action: Delegate work to co-worker
Action Input: {"coworker": "Financial Forecaster", "task": "Create a quick forecast for CAP.PA price next quarter focusing on key metrics only", "context": "The forecast should include key support/resistance levels and be as concise as possible."}


> Entering new CrewAgentExecutor chain...
I should gather key metrics and current market conditions to create a concise forecast for CAP.PA price next quarter.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/cap.pa"} 

marketwatch.comPlease enable JS and disable any ad blocker

Final Answer: marketwatch.comPlease enabl

In [16]:
from IPython.display import Markdown
Markdown(result)

Based on the information gathered from various sources, including the current trends, analyst forecasts, and fundamental indicators for Capgemini SE (CAP.PA), the recommended portfolio allocation strategy for CAP.PA with a budget of 250,000 and medium risk tolerance is as follows:

1. Equities: Allocate 60% of the portfolio budget to a diversified mix of high-quality equities, including technology, healthcare, and consumer goods sectors. This allocation aims to capitalize on the expected rise in Capgemini SE stock price in the next 3 months and the company's excellent profit margin.

2. Bonds: Allocate 30% of the portfolio budget to investment-grade bonds to provide stability and income generation. This allocation helps mitigate market risk and provides a steady stream of returns.

3. Cash: Keep 10% of the portfolio budget in cash or cash equivalents to take advantage of any investment opportunities that may arise and provide liquidity in case of market downturns.

By following this allocation strategy, CAP.PA can optimize its investment portfolio to achieve growth while managing risk effectively in the current market environment.